# 深度学习中的泛化


在 :numref:`chap_regression` 和 :numref:`chap_classification` 中，我们通过将线性模型拟合到训练数据来解决回归和分类问题。在这两种情况下，我们都提供了实用的算法来找到使观察到的训练标签似然性最大化的参数。然后，在每一章的最后，我们回忆起拟合训练数据只是一个中间目标。我们一直以来真正的追求是发现*一般模式*，基于这些模式，即使是对从同一基础总体中抽取的新样本，我们也能够做出准确的预测。机器学习研究人员是优化算法的*消费者*。有时，我们甚至必须开发新的优化算法。但归根结底，优化只是达到目的的一种手段。本质上，机器学习是一门统计学科，我们希望优化训练损失仅在于某些统计原理（已知或未知）导致生成的模型能够超越训练集进行泛化。


好的一面是，事实证明，通过随机梯度下降训练的深度神经网络在跨越无数预测问题上表现出惊人的泛化能力，包括计算机视觉；自然语言处理；时间序列数据；推荐系统；电子健康记录；蛋白质折叠；视频游戏和棋盘游戏中价值函数的近似；以及许多其他领域。不利的一面是，如果你正在寻找关于优化故事（为什么我们可以将它们拟合到训练数据）或泛化故事（为什么由此产生的模型可以泛化到未见过的例子）的直接解释，那么你可能需要给自己倒一杯饮料。虽然我们优化线性模型的过程及其解的统计性质都由一个全面的理论体系很好地描述了，但我们对深度学习的理解在这两方面仍然像西部拓荒一样。

深度学习的理论与实践都在迅速发展，理论家们采用了新策略来解释发生了什么，同时从业者继续以惊人的速度创新，建立了训练深层网络的经验法则库以及一系列直觉和民间知识，为决定在何种情况下应用哪些技术提供指导。

目前的总结是，深度学习理论已经产生了有希望的研究方向和分散的有趣结果，但距离对（i）为什么我们能够优化神经网络以及（ii）通过梯度下降学到的模型如何能够在高维任务上如此好地泛化给出全面解释还有很远的距离。然而，在实践中，（i）很少成为问题（我们总能找到适合所有训练数据的参数），因此理解泛化是一个更大的问题。另一方面，即使没有连贯的科学理论的支持，从业者也已经开发出了一整套技术，这些技术可能帮助你在实践中产生泛化良好的模型。尽管任何简短的总结都无法充分涵盖深度学习中泛化这个广泛的话题，而且整体研究状态还远未解决，但我们希望在这一节中呈现关于研究和实践状态的广泛概述。


## 重新审视过拟合与正则化

根据 :citet:`wolpert1995no` 的“没有免费午餐”定理，任何学习算法在某些分布的数据上泛化得更好，在其他分布的数据上则更差。因此，给定有限的训练集，模型依赖于某些假设：为了达到人类水平的表现，识别反映人类思考世界的*归纳偏置*可能是有用的。这样的归纳偏置显示了对具有某些属性的解决方案的偏好。例如，一个深层多层感知器有一个归纳偏置，即通过组合较简单的函数来构建复杂的函数。

随着机器学习模型编码归纳偏置，我们训练它们的方法通常包括两个阶段：(i) 拟合训练数据；(ii) 通过在保留数据上评估模型来估计*泛化误差*（基础总体上的真实误差）。我们在训练数据上的拟合与测试数据上的拟合之间的差异称为*泛化差距*，当这个差距很大时，我们说我们的模型*过拟合*了训练数据。在过拟合的极端情况下，我们可能会完全拟合训练数据，即使测试误差仍然显著。而在经典观点中，解释是我们的模型太复杂了，需要减少特征数量、非零参数的数量或参数的大小。回想一下 :numref:`sec_generalization_basics` 中的模型复杂度与损失对比图（:numref:`fig_capacity_vs_error`）。


然而，深度学习以反直觉的方式复杂化了这一画面。首先，对于分类问题，我们的模型通常有足够的表达力来完美地拟合每个训练示例，即使是在包含数百万个样本的数据集中 :cite:`zhang2021understanding`。在经典图景中，我们可能会认为这种设置位于模型复杂度轴的最右端，并且任何改进泛化误差的方法都必须通过正则化实现，要么减少模型类的复杂度，要么施加惩罚，严格限制参数可能取值的集合。但这就是事情开始变得奇怪的地方。

奇怪的是，对于许多深度学习任务（例如，图像识别和文本分类），我们通常选择的模型架构都能够实现任意低的训练损失（和零训练误差）。因为所有考虑中的模型都能实现零训练误差，*进一步提高的唯一途径就是减少过拟合*。更奇怪的是，尽管完美地拟合了训练数据，我们实际上可以通过让模型*更具表现力*来*降低泛化误差*，例如增加层数、节点数或训练更多轮次。甚至更奇怪的是，模型复杂度（如网络的深度或宽度所捕捉到的）与泛化差距之间的关系可以是非单调的，一开始更大的复杂度会带来伤害，但随后会在所谓的“双下降”模式中有所帮助 :cite:`nakkiran2021deep`。因此，深度学习实践者拥有一个技巧包，其中一些看似限制了模型，另一些则看似使其更具表现力，所有这些在某种意义上都是为了减轻过拟合而应用的。

使事情更加复杂的是，尽管经典学习理论提供的保证即使对于经典模型来说也可能过于保守，但它们似乎无力解释为什么深度神经网络一开始就具有泛化能力。由于深度神经网络能够拟合任意标签，即使是对于大型数据集，并且尽管使用了熟悉的方法如 $\ell_2$ 正则化，传统的基于复杂度的泛化界，例如基于假设类的VC维度或Rademacher复杂度，无法解释为什么神经网络能够泛化。

## 来自非参数方法的启示

初次接触深度学习时，很容易将其视为参数模型。毕竟，这些模型确实有数百万个参数。当我们更新模型时，我们更新它们的参数。当我们保存模型时，我们将它们的参数写入磁盘。然而，数学和计算机科学充满了反直觉的观点变化，以及看似不同问题之间令人惊讶的同构。虽然神经网络显然*有*参数，在某些方面，将它们视为行为类似于非参数模型可能更有成效。那么，究竟是什么使得一个模型成为非参数模型呢？虽然这个名字涵盖了多种不同的方法，但一个共同的主题是非参数方法往往随着可用数据量的增长而增长复杂度。

也许最简单的非参数模型例子是 $k$-最近邻算法（我们将在后面的章节中介绍更多的非参数模型，例如 :numref:`sec_attention-pooling`）。在这里，在训练时，学习者只需记住数据集。然后，在预测时，当遇到一个新的点 $\mathbf{x}$ 时，学习者查找 $k$ 个最近邻（那些最小化某个距离 $d(\mathbf{x}, \mathbf{x}_i')$ 的 $k$ 个点 $\mathbf{x}_i'$）。当 $k=1$ 时，这个算法被称为 $1$-最近邻，该算法总是能达到零训练误差。但这并不意味着该算法不会泛化。事实上，在一些温和的条件下，$1$-最近邻算法是一致的（最终收敛到最优预测器）。

注意，$1$-最近邻要求我们指定某个距离函数 $d$，或者等效地，指定用于特征化数据的向量值基函数 $\phi(\mathbf{x})$。对于任何选择的距离度量，我们都会实现零训练误差并最终达到最优预测器，但是不同的距离度量 $d$ 编码了不同的归纳偏置，对于有限的数据量会产生不同的预测器。不同距离度量 $d$ 的选择代表了对底层模式的不同假设，不同预测器的性能将取决于这些假设与观测数据的兼容程度。

在某种程度上，由于神经网络是过度参数化的，拥有的参数远远超过了拟合训练数据所需的数量，它们倾向于*插值*训练数据（完美拟合它），因此在某些方面表现得更像非参数模型。最近的理论研究建立了大型神经网络与非参数方法之间的深刻联系，特别是核方法。特别地，:citet:`Jacot.Grabriel.Hongler.2018` 展示了在极限情况下，当具有随机初始化权重的多层感知机变得无限宽时，它们等价于（非参数的）核方法，对于特定的选择核函数（基本上是一种距离函数），他们称之为神经切线核。虽然当前的神经切线核模型可能不能完全解释现代深度网络的行为，但作为分析工具的成功强调了非参数建模在理解过参数化深度网络行为方面的有用性。

## 提前停止

虽然深度神经网络有能力拟合任意标签，即使标签被错误地或随机地分配 :cite:`zhang2021understanding`，但这种能力只有在经过多次迭代训练后才会显现。一条新的工作线 :cite:`Rolnick.Veit.Belongie.Shavit.2017` 揭示了在标签噪声的情况下，神经网络倾向于首先拟合干净标记的数据，然后才插值误标的数据。此外，已经确定这种现象直接转化为泛化的保证：每当模型拟合了干净标记的数据但还没有拟合训练集中包含的随机标记示例时，它实际上就已经泛化了 :cite:`Garg.Balakrishnan.Kolter.Lipton.2021`。

这些发现共同帮助激励了*提前停止*，这是一种经典的深度神经网络正则化技术。这里，不是直接约束权重的值，而是约束训练的轮数。最常见的确定停止标准的方法是在整个训练过程中监控验证误差（通常每轮检查一次），并在验证误差连续几个轮次内没有减少超过某个小量 $\epsilon$ 时切断训练。这有时被称为*耐心准则*。除了在标签噪声情况下有可能带来更好的泛化外，提前停止的另一个好处是节省了时间。一旦满足耐心准则，就可以终止训练。对于可能需要几天时间在八个或更多 GPU 上同时训练的大模型，调优良好的提前停止可以为研究人员节省几天的时间，并为他们的雇主节省数千美元。

值得注意的是，当没有标签噪声并且数据集是*可实现的*（类别确实是可区分的，例如区分猫和狗）时，提前停止通常不会带来泛化的显著改进。另一方面，当存在标签噪声或标签内在变异性（例如，预测患者的死亡率）时，提前停止至关重要。直到模型插值噪声数据为止进行训练通常是不好的主意。

## 深度网络的经典正则化方法

在 :numref:`chap_regression` 中，我们描述了几种经典正则化技术，用于约束我们模型的复杂性。特别是在 :numref:`sec_weight_decay` 中介绍了一种称为权重衰减的方法，它通过在损失函数中添加一个正则化项来惩罚权重的大值。根据惩罚哪种权重范数，这种方法被称为岭正则化（对于 $\ell_2$ 处罚）或lasso正则化（对于 $\ell_1$ 处罚）。在这些正则化的经典分析中，它们被认为是足够限制权重可以取的值，以防止模型拟合任意标签。

在深度学习实现中，权重衰减仍然是一个流行的工具。然而，研究人员注意到典型的 $\ell_2$ 正则化强度不足以防止网络插值数据 :cite:`zhang2021understanding`，因此如果将其解释为正则化的好处，可能只在结合提前停止标准时才有意义。如果没有提前停止，就像层数或节点数（在深度学习中）或距离度量（在1-最近邻中）一样，这些方法可能带来更好的泛化，不是因为它们有意义地限制了神经网络的能力，而是因为它们以某种方式编码了与感兴趣的数据集中的模式更兼容的归纳偏置。因此，即使其有效性的理论依据可能大不相同，经典正则化器在深度学习实现中仍然流行。

值得注意的是，深度学习研究人员也借鉴了最初在经典正则化背景下流行的技术，例如向模型输入添加噪声。在下一节中，我们将介绍著名的dropout技术（由 :citet:`Srivastava.Hinton.Krizhevsky.ea.2014` 发明），它已成为深度学习的主要内容，尽管其有效的理论基础同样神秘。

## 总结

与倾向于参数少于样本的经典线性模型不同，深度网络往往是过度参数化的，并且对于大多数任务都能够完美地拟合训练集。这种*插值机制*挑战了许多根深蒂固的直觉。功能上，神经网络看起来像参数模型。但将它们视为非参数模型有时可以提供更可靠的直觉。因为通常情况下，所有考虑中的深度网络都能够拟合所有的训练标签，几乎所有增益都必须通过缓解过拟合（缩小*泛化差距*）来实现。矛盾的是，减少泛化差距的干预措施有时似乎增加了模型复杂度，有时又似乎降低了复杂度。然而，这些方法很少将复杂度降低到足以用经典理论解释深度网络的泛化，而*为什么某些选择会导致泛化改善*仍然是一个巨大的开放问题，尽管许多杰出研究人员做出了共同努力。

## 练习

1. 传统基于复杂度的度量在哪些方面无法解释深度神经网络的泛化？
1. 为什么*提前停止*可以被视为一种正则化技术？
1. 研究人员通常如何确定停止标准？
1. 在什么重要因素上，早期停止能大幅提高泛化的案例与其他情况有所不同？
1. 除了泛化之外，描述早期停止的另一个好处。

[讨论](https://discuss.d2l.ai/t/7473)